In [21]:
import os
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.s3 import s3_path_join
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer


import sagemaker
import threading
import time
import matplotlib.pyplot as plt

#SageMaker Role
ROLE = os.getenv('SAGEMAKER_ROLE')
# Hub Model configuration. https://huggingface.co/models
hub = {
    'HF_MODEL_ID':'unitary/unbiased-toxic-roberta',
    'HF_TASK':'text-classification'
}
sagemaker_session_bucket = 'sagemaker-godeltech'
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker bucket: sagemaker-godeltech
sagemaker session region: eu-west-1


In [50]:
repository="unitary/unbiased-toxic-roberta"
model_id = "unbiased-toxic-roberta"

In [3]:
%cd ../tmp && git clone https://huggingface.co/$repository

Cloning into 'unbiased-toxic-roberta'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 37 (delta 16), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (37/37), 542.02 KiB | 827.00 KiB/s, done.
Filtering content: 100% (2/2), 951.16 MiB | 17.98 MiB/s, done.


In [42]:
%cd $model_id
%cp -r ../godel_demo/aux/inference.py ./code/inference.py
!tar zcvf model.tar.gz *

code/
code/.ipynb_checkpoints/
code/.ipynb_checkpoints/inference-checkpoint.py
code/inference.py
config.json
flax_model.msgpack
merges.txt
pytorch_model.bin
README.md
special_tokens_map.json
tokenizer_config.json
vocab.json


In [43]:
cust_model_url = sess.upload_data(
    path="./model.tar.gz",
    key_prefix="transformers/cust_model",
)
cust_model_url

's3://sagemaker-godeltech/transformers/cust_model/model.tar.gz'

In [44]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=cust_model_url,    # path to your model and script
   role=ROLE,                    # iam role with permissions to create an Endpoint
   transformers_version="4.12",  # transformers version used
   pytorch_version="1.9",        # pytorch version used
   py_version='py38',            # python version used
)

# deploy the endpoint endpoint
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge",
    endpoint_name=f"toxicbert-custom-endpoint"
    )

----------!

In [48]:
kayne_west_tweet = """
I’m a bit sleepy tonight but when I wake up I’m going death con 3 On JEWISH PEOPLE.
The funny thing is I actually can’t be Anti Semitic because black people are actually Jew 
also You guys have toyed with me and tried to black ball anyone whoever opposes your agenda.
"""

predictor.predict({
	'inputs': kayne_west_tweet
})

{'toxicity': '0.5768711',
 'severe_toxicity': '0.0011559735',
 'obscene': '0.007310892',
 'identity_attack': '0.50905406',
 'insult': '0.08191899',
 'threat': '0.013708219',
 'sexual_explicit': '0.00036571428',
 'male': '0.0065239645',
 'female': '0.00083787285',
 'homosexual_gay_or_lesbian': '0.0011106308',
 'christian': '0.0031713771',
 'jewish': '0.99794334',
 'muslim': '0.0031509597',
 'black': '0.9684012',
 'white': '0.005044794',
 'psychiatric_or_mental_illness': '0.001307617'}

In [49]:
predictor.delete_model()
predictor.delete_endpoint()